In [ ]:
from tensorflow import keras
models = []
model_names = ['densenet201', 'inception_v3', 'mobilenet_v2', 'resnet50_v2', 'vgg19', 'xception']
models.append(keras.models.load_model('/content/drive/Shareddrives/ANC/models/models_phase_2/models/densenet201.h5'))
models.append(keras.models.load_model('/content/drive/Shareddrives/ANC/models/models_phase_2/models/inception_v3.h5'))
models.append(keras.models.load_model('/content/drive/Shareddrives/ANC/models/models_phase_2/models/mobilenet_v2.h5'))
models.append(keras.models.load_model('/content/drive/Shareddrives/ANC/models/models_phase_2/models/resnet50_v2.h5'))
models.append(keras.models.load_model('/content/drive/Shareddrives/ANC/models/models_phase_2/models/vgg19.h5'))
models.append(keras.models.load_model('/content/drive/Shareddrives/ANC/models/models_phase_2/models/  .h5'))

In [ ]:
#Generate sequence
model_sequence = []
model_len = len(models)

for p1 in range(0, model_len-2):
  for p2 in range(p1+1, model_len-1):
    for p3 in range(p2+1, model_len):
      model_sequence.append([p1, p2, p3])

# print(model_sequence)

In [ ]:
import numpy as np 
import tensorflow as tf
from collections import defaultdict 

image_size = 224
classes = ["COVID_19 +ve","COVID_19 -ve"]

def ensemble(x, weights, models): 
    '''
    returns a weighted average of predictions made by the models\n
    x -> input image \n
    weights -> a list of weights \n
    models -> a list of models\n    
    '''      
    outputs = []    
    for model in models:
        outputs.append(list(model.predict(x)[0]))                
    
    outputs = np.array(outputs)
    avg = np.average(a=outputs,axis=0,weights=weights)
    return avg


def equal(pred, label):
  pred_id = np.argmax(pred)
  if (pred_id == label):
    return True
  else:
    return False


def accuracy(predicted_values, y_truths):
  '''
  returns accuracy\n
  predicted_values = a numpy array containing the predictions\n
  y_truths = a numpy array containing the truth values\n  
  '''
  total = len(y_truths)
  correct = 0
  for i in range(len(y_truths)):
    if equal(predicted_values[i],y_truths[i]):
      correct += 1
  acc = correct / total
  return acc


def generate_weights(x_val, y_val, models):
    '''
    returns a list of weights
    '''
    print(d)
    accuracy = []
    weights = np.full((1,len(models)), 100.0)
    for model in models:
      if(str(model) not in d):
        d[str(model)] = model.evaluate(x_val, y_val)[1]
      accuracy.append(100*d[str(model)])
    weights = weights - accuracy
    weights = weights**2
    sum = np.sum(weights)
    weights = weights/sum
    weights = 1/weights
    weights = weights**2
    sum = np.sum(weights)
    weights = weights/sum
    return weights

In [ ]:
%cd '/content/drive/Shareddrives/ANC/dataset/processed_images/Dataset_output_new/'

/content/drive/.shortcut-targets-by-id/1rXHszhdZVOaGafItC_bx5jPEoHxGPOBq/Dataset_ouput_new


In [ ]:
import numpy as np

# x_test = np.load('New_Btest_images.npy')
# y_test = np.load('New_Btest_labels.npy')

x_test = np.load('New_Btest_images.npy')
y_test = np.load('New_Btest_labels.npy')

In [ ]:
from tqdm import tqdm_notebook
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf 
import numpy as np

model_considered = []
generated_weights_sequence = []
output_pred = []
confusion_matrices = []
accuracies = []

for sequence in tqdm_notebook(model_sequence):
  model_considered = []

  for index in sequence:
    model_considered.append(models[index])

  print("\n\n---------------------------------------------")
  print(f"Models considered for ensemble: {model_names[sequence[0]]} ; {model_names[sequence[1]]} ; {model_names[sequence[2]]}")
  print("---------------------------------------------\n")
  
  # Generating weights
  print("Generating weights...")
  w = generate_weights(x_test, y_test, model_considered)[0]
  print(f"Weights: {w}")
  generated_weights_sequence.append(w)

  # average_val, pred_outputs = ensemble(x_test, generated_weights, model_considered)
  # print(f"Average value: {average_val}")
  # print(f"Predicted output: {pred_outputs}")
  predictions = []

  print("Performing Ensemble of the 3 models...")
  for i in tqdm_notebook(range(len(x_test))):
    pred = ensemble(x_test[i].reshape(-1,image_size,image_size,3),w,model_considered)
    predictions.append(pred)
  print(f"Predictions: {predictions[:3]}")
  print(f"Test data: {y_test[:3]}")
  # output_pred.append((average_val, pred_outputs))
  print("---------------------------------------------\n")

  print("Accuracy: ",accuracy(predictions,y_test))
  accuracies.append(accuracy(predictions,y_test))

  y_pred = np.argmax(np.array(predictions), axis=1)
  print(f"Predicted y: {y_pred[:3]}")
  # y_pred = [np.argmax(prediction[0]) for prediction in predictions]

  print("The classification report: ")
  print(classification_report(y_pred=y_pred, y_true=y_test))
  print("Confusion Matrix: ")
  print(confusion_matrix(y_pred=y_pred, y_true=y_test))
  confusion_matrices.append(confusion_matrix(y_pred=y_pred, y_true=y_test))
  print("---------------------------------------------\n")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/20 [00:00<?, ?it/s]



---------------------------------------------
Models considered for ensemble: densenet201 ; inception_v3 ; mobilenet_v2
---------------------------------------------

Generating weights...
defaultdict(<class 'str'>, {'<keras.engine.functional.Functional object at 0x7fa9dac768d0>': 0.9644444584846497, '<keras.engine.functional.Functional object at 0x7fa9da362e90>': 0.9733333587646484, '<keras.engine.functional.Functional object at 0x7fa9d466d610>': 0.1111111119389534, '<keras.engine.functional.Functional object at 0x7fa9de8e3410>': 0.9555555582046509, '<keras.engine.functional.Functional object at 0x7fa9d41cab90>': 0.9155555367469788, '<keras.engine.functional.Functional object at 0x7fa9d3f6e6d0>': 0.9866666793823242})
Weights: [2.40355527e-01 7.59643858e-01 6.15309180e-07]
Performing Ensemble of the 3 models...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/225 [00:00<?, ?it/s]

...

Predictions: [array([1.41432963e-06, 9.99998609e-01]), array([1.31534689e-05, 9.99986860e-01]), array([3.28896010e-08, 9.99999968e-01])]
Test data: [1 1 1]
---------------------------------------------

Accuracy:  0.9866666666666667
Predicted y: [1 1 1]
The classification report: 
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        25
           1       0.99      0.99      0.99       200

    accuracy                           0.99       225
   macro avg       0.97      0.96      0.97       225
weighted avg       0.99      0.99      0.99       225

Confusion Matrix: 
[[ 23   2]
 [  1 199]]
---------------------------------------------



---------------------------------------------
Models considered for ensemble: mobilenet_v2 ; vgg19 ; xception
---------------------------------------------

Generating weights...
defaultdict(<class 'str'>, {'<keras.engine.functional.Functional object at 0x7fa9dac768d0>': 0.9644444584846497, '<keras.

  0%|          | 0/225 [00:00<?, ?it/s]

Predictions: [array([3.98096654e-05, 9.99960215e-01]), array([3.05611510e-06, 9.99996957e-01]), array([5.20317696e-08, 9.99999948e-01])]
Test data: [1 1 1]
---------------------------------------------

Accuracy:  0.9866666666666667
Predicted y: [1 1 1]
The classification report: 
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        25
           1       0.99      0.99      0.99       200

    accuracy                           0.99       225
   macro avg       0.97      0.96      0.97       225
weighted avg       0.99      0.99      0.99       225

Confusion Matrix: 
[[ 23   2]
 [  1 199]]
---------------------------------------------



---------------------------------------------
Models considered for ensemble: resnet50_v2 ; vgg19 ; xception
---------------------------------------------

Generating weights...
defaultdict(<class 'str'>, {'<keras.engine.functional.Functional object at 0x7fa9dac768d0>': 0.9644444584846497, '<keras.e

  0%|          | 0/225 [00:00<?, ?it/s]

Predictions: [array([4.04741764e-05, 9.99959550e-01]), array([1.61135415e-05, 9.99983899e-01]), array([3.33068007e-08, 9.99999968e-01])]
Test data: [1 1 1]
---------------------------------------------

Accuracy:  0.9866666666666667
Predicted y: [1 1 1]
The classification report: 
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        25
           1       0.99      0.99      0.99       200

    accuracy                           0.99       225
   macro avg       0.97      0.96      0.97       225
weighted avg       0.99      0.99      0.99       225

Confusion Matrix: 
[[ 23   2]
 [  1 199]]
---------------------------------------------



In [ ]:
accuracy_indices = [i for i, accuracy in enumerate(accuracies) if accuracy == 0.99]

print(accuracy_indices)

[3, 6, 8, 9, 12, 14, 15, 17, 18, 19]


In [ ]:
print(accuracies)

[0.98, 0.98, 0.98, 0.99, 0.96, 0.96, 0.99, 0.96, 0.99, 0.99, 0.97, 0.97, 0.99, 0.97, 0.99, 0.99, 0.96, 0.99, 0.99, 0.99]


In [ ]:
model_list = []

for i in accuracy_indices:
  sequence = model_sequence[i]
  temp_list = []
  for ind in sequence:
    temp_list.append(model_names[ind])
  print(temp_list)
  model_list.append(temp_list)
  
# print(model_list)

['densenet201', 'inception_v3', 'xception']
['densenet201', 'mobilenet_v2', 'xception']
['densenet201', 'resnet50_v2', 'xception']
['densenet201', 'vgg19', 'xception']
['inception_v3', 'mobilenet_v2', 'xception']
['inception_v3', 'resnet50_v2', 'xception']
['inception_v3', 'vgg19', 'xception']
['mobilenet_v2', 'resnet50_v2', 'xception']
['mobilenet_v2', 'vgg19', 'xception']
['resnet50_v2', 'vgg19', 'xception']


In [ ]:
print(generated_weights_sequence)


[array([2.40355527e-01, 7.59643858e-01, 6.15309180e-07]), array([0.21881358, 0.6915605 , 0.08962592]), array([0.23855355, 0.75394871, 0.00749774]), array([0.01827201, 0.05774871, 0.92397928]), array([7.09420100e-01, 1.81611260e-06, 2.90578083e-01]), array([9.69525331e-01, 2.48198094e-06, 3.04721875e-02]), array([1.93918646e-02, 4.96430952e-08, 9.80608086e-01]), array([0.69394834, 0.28424086, 0.0218108 ]), array([0.01923905, 0.00788031, 0.97288064]), array([1.93800537e-02, 6.09115214e-04, 9.80010831e-01]), array([8.85268850e-01, 7.17065036e-07, 1.14730433e-01]), array([9.90152499e-01, 8.02020468e-07, 9.84669854e-03]), array([5.88235266e-02, 4.76468750e-08, 9.41176426e-01]), array([0.87754387, 0.11372928, 0.00872685]), array([0.05837848, 0.00756582, 0.93405569]), array([5.87891391e-02, 5.84636135e-04, 9.40626225e-01]), array([5.80455810e-06, 9.28729513e-01, 7.12646822e-02]), array([5.02180399e-08, 8.03488826e-03, 9.91965062e-01]), array([5.05933587e-08, 6.21153165e-04, 9.99378796e-01]), 